# A Nonlinear conservation law: the shallow water equation in 2D

We consider the shallow water equations as an example of a nonlinear conservation law, i.e. we consider

$$
  \partial_t \mathbf{U} + \operatorname{div}(\mathbf{F} (\mathbf{U} )) = 0 \qquad in \qquad \Omega \times[0,T],
$$
with 
$$
\mathbf{U} = (h, \mathbf{w}) = (h, hu, hv) = (\mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3)
$$
and 
$$
  \mathbf{F}(\mathbf{U}) = \left( \begin{array}{c@{}c@{\qquad}c@{}c} h u & & h v & \\ h u^2 &+ \frac12 g h^2 & h u v & \\ h u v &  & h v^2 & + \frac12 g h^2 \end{array} \right) 
  = \left( \begin{array}{cc} \mathbf{u}_2 & \mathbf{u}_3 \\ \frac{\mathbf{u}_2^2}{\mathbf{u}_1} + \frac12 g \mathbf{u}_1^2 & \frac{\mathbf{u}_2 \mathbf{u}_3}{\mathbf{u}_1} \\ \frac{\mathbf{u}_2 \mathbf{u}_3}{\mathbf{u}_1} & \frac{\mathbf{u}_3^2}{\mathbf{u}_1} + \frac12 g \mathbf{u}_1^2 \end{array} \right)  
  = \left( \begin{array}{cc} h \mathbf{w}^T \\ h \mathbf{w} \otimes \mathbf{w} + \frac12 g h^2 \mathbf{I} \end{array} \right)
$$

## Jacobian of the flux for shallow water:

$$
\begin{align*}
\mathbf{A}_1 & =
\left(
      \begin{array}{ccc}
      0 & 1 & 0 \\
      - \frac{\mathbf{u}_2^2}{\mathbf{u}_1^2} + g \mathbf{u}_1 & 2 \frac{\mathbf{u}_2}{\mathbf{u}_1} & 0 \\
      - \frac{\mathbf{u}_2 \mathbf{u}_3}{\mathbf{u}_1^2} & \frac{\mathbf{u}_3}{\mathbf{u}_1} & \frac{\mathbf{u}_2}{\mathbf{u}_1}
      \end{array}
      \right)
=
\left(
      \begin{array}{ccc}
      0 & 1 & 0 \\
      - u^2 + g h & 2 u & 0 \\
      - u v & v & u
      \end{array}
      \right) \\
\mathbf{A}_2 & =
\left(
      \begin{array}{ccc}
      0 & 0 & 1 \\
      - \frac{\mathbf{u}_2\mathbf{u}_3}{\mathbf{u}_1^2} & \frac{\mathbf{u}_3}{\mathbf{u}_1} & \frac{\mathbf{u}_2}{\mathbf{u}_1} \\
      - \frac{\mathbf{u}_3^2}{\mathbf{u}_1^2} + g \mathbf{u}_1  & 0 & 2\frac{\mathbf{u}_3}{\mathbf{u}_1}
      \end{array}
      \right)
=
\left(
      \begin{array}{ccc}
      0 & 0 & 1 \\
      - uv & v & u \\
      - v^2 + gh & 0 & 2 v
      \end{array}
      \right) \\
\mathbf{A}(\mathbf{u}, \mathbf{n}) & = n_1 \mathbf{A}_1 + n_2 \mathbf{A}_2 =
\left(
      \begin{array}{ccc}
      0 & n_1 & n_2 \\
      - u \alpha - g h n_1  & \alpha + u n_1 & u n_2 \\
      - v \alpha - g h n_2  & v n_1 & \alpha + v n_2
      \end{array}
      \right), \quad \text{ with } \alpha = (\mathbf{u},\mathbf{n}),
\end{align*}
$$

spectrum:

$$
\rho( \mathbf{A}(\mathbf{u}, \mathbf{n}) ) = \{ \alpha + \sqrt{g h}, \alpha, \alpha - \sqrt{g h} \} 
$$


### Notes:
h seems to represent mass, since linear momentum is hvx, hvy (also represented as hu, hv)
and velocity is vy, vy (also represented as vu, vv).  

I don't see how h could be height, for example we see below: vx = hvx/h

Basically we have two domains 1 and 2 with an overall boundary 'wall' and a boundary 'dam' that separates domains 1 and 2.  domain 1 is on the left, with material value 10.  domain 2 has material value 2.

In [1]:
from ngsolve import * # sloppy
import netgen.gui
%gui tk

### The dam break problem (geometry)

<center> ![alt](geomsketch.png) </center>

In [2]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()

pnts =[ (-12,-5), (-7,-5), (-5,-5), (-3,-5), (-3,-3), 
        (-3,-1), (-1,-1), ( 0,-1), ( 1,-1), ( 3,-1), 
        ( 3,-3), ( 3,-5), ( 5,-5), ( 7,-5), ( 12,-5), 
        ( 12, 5), ( 7, 5), ( 5, 5), ( 3, 5), ( 3, 3), 
        ( 3, 1), ( 1, 1), ( 0, 1), (-1, 1), (-3, 1), 
        (-3, 3), (-3, 5), (-5, 5), (-7, 5), (-12, 5)]
pnts = [geo.AppendPoint(*pnt) for pnt in pnts]

In [3]:
curves = [[["line",0,1],"wall",1, 0], [["line",1,2],"wall",1, 0],
          [["spline3",2,3,4],"wall",1, 0],[["spline3",4,5,6],"wall",1, 0],[["line",6,7],"wall",1, 0],
          [["line",7,22],"dam",1, 2],     # <--- dam interface
          [["line",7,8],"wall",2, 0],[["spline3",8,9,10],"wall",2, 0],
          [["spline3",10,11,12],"wall",2, 0],[["line",12,13],"wall",2, 0],[["line",13,14],"wall",2, 0],
          [["line",14,15],"wall",2, 0], # <--- right boundary
          [["line",15,16],"wall",2, 0],[["line",16,17],"wall",2, 0],
          [["spline3",17,18,19],"wall",2, 0],[["spline3",19,20,21],"wall",2, 0],
          [["line",21,22],"wall",2, 0],[["line",22,23],"wall",1, 0],
          [["spline3",23,24,25],"wall",1, 0],[["spline3",25,26,27],"wall",1, 0],
          [["line",27,28],"wall",1, 0],[["line",28,29],"wall",1, 0],
          [["line",29,0],"wall",1, 0]]   # <--- left boundary

for c,bc,l,r in curves:
    geo.Append(c,bc=bc,leftdomain=l, rightdomain=r) 
geo.SetMaterial(1,"upperlevel")
geo.SetMaterial(2,"lowerlevel")    

In [4]:
mesh = Mesh(geo.GenerateMesh(maxh=2))
mesh.Curve(3)
Draw(mesh)

### Approximation space

In [5]:
order = 2
fes = L2(mesh,order=order,dim=3)

### initial and boundary conditions

In [6]:
U,V = fes.TnT() # "Trial" and "Test" function
h, hu, hv = U

# initial conditions
h0mat = {"upperlevel" : 10, "lowerlevel" : 2}
U0 = CoefficientFunction((CoefficientFunction([h0mat[mat] for mat in mesh.GetMaterials()]),0,0))

# boundary conditions
hbndreg = CoefficientFunction([{"wall" : h, "dam" : 0}[rg] for rg in mesh.GetBoundaries()])
hubndreg = CoefficientFunction([{"wall" : -hu, "dam" : 0}[rg] for rg in mesh.GetBoundaries()])
hvbndreg = CoefficientFunction([{"wall" : -hv, "dam" : 0}[rg] for rg in mesh.GetBoundaries()])

Ubnd = CoefficientFunction((hbndreg,hubndreg,hvbndreg))

# constant for gravitational force
g=1

### Flux definition and numerical flux choice (Lax-Friedrich)

$$
  \mathbf{F}(\mathbf{U}) = \left( \begin{array}{c@{}c@{\qquad}c@{}c} h u & & h v & \\ h u^2 &+ \frac12 g h^2 & h u v & \\ h u v &  & h v^2 & + \frac12 g h^2 \end{array} \right) 
  = \left( \begin{array}{cc} \mathbf{u}_2 & \mathbf{u}_3 \\ \frac{\mathbf{u}_2^2}{\mathbf{u}_1} + \frac12 g \mathbf{u}_1^2 & \frac{\mathbf{u}_2 \mathbf{u}_3}{\mathbf{u}_1} \\ \frac{\mathbf{u}_2 \mathbf{u}_3}{\mathbf{u}_1} & \frac{\mathbf{u}_3^2}{\mathbf{u}_1} + \frac12 g \mathbf{u}_1^2 \end{array} \right)  
  = \left( \begin{array}{cc} h \mathbf{w}^T \\ h \mathbf{w} \otimes \mathbf{w} + \frac12 g h^2 \mathbf{I} \end{array} \right)
$$

In [7]:
def F(U):
    h, hvx, hvy = U
    vx = hvx/h
    vy = hvy/h
    return CoefficientFunction(((hvx,hvy),
                                (hvx*vx + 0.5*g*h**2, hvx*vy),
                                (hvx*vy, hvy*vy + 0.5*g*h**2)),dims=(3,2))

$$
  \hat{\mathbf{F}}(\mathbf{U}) \cdot \mathbf{n} = \frac{(\mathbf{F}(\mathbf{U}^l)+\mathbf{F}(\mathbf{U}^r)}{2} \cdot \mathbf{n} + \frac{|\lambda|}{2} (\mathbf{U}^l - \mathbf{U}^r), \quad (\cdot^l: \text{current element}, \quad \cdot^r: \text{neighbor element})
$$

In [8]:
n = specialcf.normal(mesh.dim)

def Max(u,v):
    return IfPos(u-v,u,v)
def Fmax(A,B):
    ha, hua, hva = A
    hb, hub, hvb = B
    vnorma = sqrt(hua**2+hva**2)/ha
    vnormb = sqrt(hub**2+hvb**2)/hb
    return Max(vnorma+sqrt(g*A[0]),vnormb+sqrt(g*B[0]))

def Fhatn(U):
    Uhat = U.Other(bnd=Ubnd)
    return (0.5*F(U)+0.5*F(Uhat))*n + Fmax(U,Uhat)/2*(U-Uhat)

### DG formulation

In [9]:
def DGBilinearForm(fes,F,Fhatn,Ubnd):
    a = BilinearForm(fes, nonassemble=True)    
    a += SymbolicBFI ( (- InnerProduct(F(U),grad(V).trans)).Compile())
    a += SymbolicBFI ( (InnerProduct(Fhatn(U),V)).Compile(), element_boundary=True)
    return a

a = DGBilinearForm(fes,F,Fhatn,Ubnd)

#### Simple fix to deal with shocks: artificial diffusion:

In [10]:
from DGdiffusion import AddArtificialDiffusion

artvisc = Parameter(1.0)
if order > 0:
    AddArtificialDiffusion(a,Ubnd,artvisc,compile=True)

ModuleNotFoundError: No module named 'DGdiffusion'

### Visualization of solution quantities

In [11]:
gfu = GridFunction(fes)
gfh, gfhu, gfhv = gfu
gfvu = gfhu/gfh
gfvv = gfhv/gfh
momentum = CoefficientFunction((gfhu,gfhv))
velocity = CoefficientFunction((gfvu,gfvv))
gfu.Set(U0)
Draw(momentum,mesh,"mom")
Draw(velocity,mesh,"vel")
Draw(gfh,mesh,"h")

### Implicit Euler time stepping

In [12]:
def TimeLoop(a,gfu,dt,T,nsamplings=100):
    #gfu.Set(U0)
    res = gfu.vec.CreateVector()
    fes = a.space
    t = 0
    nsteps = int(ceil(T/dt))    
    i = 0
    with TaskManager():
        while t <= T - 0.5*dt:
            a.Apply(gfu.vec, res)
            fes.SolveM(rho=CoefficientFunction(1),vec=res)
            gfu.vec.data -= dt * res
            t += dt
            if (i+1) % int(nsteps/nsamplings) == 0:
                Redraw(blocking=True)
            i+=1
            print("\rt=",t,end="")
    Redraw()     
TimeLoop(a,gfu,dt=0.0004,T=5)

t= 5.9779999999994075745 0.44240000000000856 0.4712000000000094 0.5580000000000038 0.6491999999999938 0.6783999999999906 0.7047999999999877 0.7319999999999847 0.78839999999997850.8435999999999724 1.019999999999953 1.1079999999999433 1.191599999999934 1.219999999999931 1.3579999999999157 1.3859999999999126 1.4135999999999096 1.4427999999999064 1.5399999999998957 1.61759999999988711.6979999999998783 1.7847999999998687 1.81239999999986572.15039999999982842.1787999999998253 2.2067999999998222 2.236799999999819 2.3843999999998027 2.453599999999795 2.4667999999997936 3.532799999999676 3.55919999999967333.7291999999996546 3.7851999999996484 3.841999999999642 3.8679999999996393 3.8979999999996363.924799999999633 3.95159999999963 3.979599999999627 4.000799999999625 4.481199999999572 4.695599999999549 4.753599999999542 4.784399999999539 4.812799999999536 4.8723999999995294.901599999999526 4.931199999999523 4.976799999999518 5.0047999999995145 5.033999999999511 5.088399999999505 5.418799999999469

KeyboardInterrupt: 

### Notes:
When I changed T from 5 to 10, the solution became unstable (infs everywhere).  Maybe this has to do with the shock referenced above.  It would be interesting to see if adding viscosity stabilizes the solution.  It seems the artificial diffusion parameter was removed from the TimeLoop function.

You may play around with this example. 
* change the artificial diffusion parameter: How does it influence the solution
* change boundary conditions: left boundary -> fixed height and non-reflecting
* change the initial heights
* introduce a (circular) obstacle below the dam
* Generate output to create a video: To this end take a look at the final unit of this section.

In [ ]:
%%HTML
<video width="1000" height="600" controls>
    <source src="shallow2D.mp4" type="video/mp4">
    # <source src="shallow2D.mov">    # Apple QuickTime 
</video>